In [1]:
# default_exp frappe.suite

# Frappe suite

> Frappe is the package which has definItions for the test suite `deescribe` and tets function `It`. 

In [2]:
#hide
from nbdev.showdoc import *
from nbdev.export import notebook2script
from fastcore.basics import *
from fastcore.test import *

In [21]:
#export
from contextlib import ContextDecorator

In [4]:
def sum():
    return 1+1

In [5]:
#export
class Describe(ContextDecorator):
    "contextmanager to describe a test suite"
    def __init__(self, desc): store_attr('desc', self)

    def __enter__(self):
        "start the execution of the suite and do setup"
        print(self.desc)
        return self
    
    def __exit__(self, type, value, traceback):
        "cleanup and exIt the suite"
        print(f'{self.desc} completed')
        print(type, value, traceback, sep="\n")
        return True

In [6]:
show_doc(Describe.__enter__)

<h4 id="Describe.__enter__" class="doc_header"><code>Describe.__enter__</code><a href="__main__.py#L6" class="source_link" style="float:right">[source]</a></h4>

> <code>Describe.__enter__</code>()

start the execution of the suite and do setup

In [7]:
show_doc(Describe.__exit__)

<h4 id="Describe.__exit__" class="doc_header"><code>Describe.__exit__</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>Describe.__exit__</code>(**`type`**, **`value`**, **`traceback`**)

cleanup and exIt the suite

In [8]:
@Describe("fake failing test")
def test():
    return test_eq(sum(),1)
    
test()

fake failing test
fake failing test completed
<class 'AssertionError'>
==:
2
1


This won't work👇

In [9]:
with Describe("fake negative test") as d:
    d.test_ne(sum(), 1)

fake negative test
fake negative test completed
<class 'AttributeError'>
'Describe' object has no attribute 'test_ne'


This works👇

In [10]:
with Describe("positive fake test suite"):
    test_eq(sum(),2)

positive fake test suite
positive fake test suite completed
None
None
None


In [11]:
with Describe("first describe"):
    print('I am inside first describe')
    with Describe("second describe") as d:
        test_eq(sum(),2)

first describe
I am inside first describe
second describe
second describe completed
None
None
None
first describe completed
None
None
None


In [12]:
@Describe("fake positive test")
def postest():
    test_eq(sum(), 2)

postest()

fake positive test
fake positive test completed
None
None
None


In [13]:
@Describe("first describe")
@Describe("second describe")
def postest():
    test_eq(sum(), 2)

postest()

first describe
second describe
second describe completed
None
None
None
first describe completed
None
None
None


In [14]:
#export
class It(ContextDecorator):
    "contextmanager to describe a test function"
    def __init__(self, desc): store_attr('desc', self)

    def __enter__(self):
        "start the execution of the test"
        print(self.desc)
        return self
    
    def __exit__(self, type, value, traceback):
        "cleanup and exit the test"
        print(f'{self.desc} completed')
        print(type, value, traceback, sep="\n")
        return True

In [15]:
with Describe("fake test suite"):
    with It("should have some fake result"):
        test_eq(sum(),2)

fake test suite
should have some fake result
should have some fake result completed
None
None
None
fake test suite completed
None
None
None


In [16]:
with Describe("Fake posItive test suite"):
    with It("should have positive test result"):
        test_eq(sum(), 2)

Fake posItive test suite
should have positive test result
should have positive test result completed
None
None
None
Fake posItive test suite completed
None
None
None


In [17]:
with Describe("Fake posItive test suite"):
    with It("should have posItive test result"):
        test_eq(sum(), 2)

    with It("should have negative test result"):
        test_ne(sum(),1)

Fake posItive test suite
should have posItive test result
should have posItive test result completed
None
None
None
should have negative test result
should have negative test result completed
None
None
None
Fake posItive test suite completed
None
None
None


In [18]:
@Describe("Fake positive test suite")
@It("should have positive test result")
def postest():
    return test_eq(sum(), 2)

postest()

Fake positive test suite
should have positive test result
should have positive test result completed
None
None
None
Fake positive test suite completed
None
None
None


In [19]:
@Describe("Fake negative test suite")
@It("should have negative test result")
def negtest():
    return test_ne(sum(), 1)

negtest()

Fake negative test suite
should have negative test result
should have negative test result completed
None
None
None
Fake negative test suite completed
None
None
None


In [20]:
notebook2script()

Converted 00_frappe.suite.ipynb.
